In [5]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import os
sys.path.append(os.path.abspath('/Users/guhansundar/Documents/GuData/ObjectiveSubjectiveHealth/'))
from scripts.parse_surveys import _resolve_survey_app_id_from_db, _prepare_wide_by_app_from_db, get_survey_data_for_subject, list_available_surveys


In [50]:
DATABASE_PATH = '/Users/guhansundar/Documents/GuData/ObjectiveSubjectiveHealth/data/database.db'
conn = sqlite3.connect(DATABASE_PATH)


In [51]:
events_df = pd.read_sql_query("SELECT * FROM events", conn)
events_df.groupby('app_id')['session_id'].nunique()

app_id
behavioral       3
daily_core       3
nback_1min_v1    3
pvt_1min_v1      2
stroop           4
wellbeing        4
Name: session_id, dtype: int64

In [ ]:
events_df['ts_utc'].apply(lambda x: pd.to_datetime(x, unit='ms'))

,id,subject_id,session_id,app_id,app_type,event_type,event_index,ts_utc,tz,payload,server_ts
0,4,demo,b238e6d7-ec06-461f-a9d4-7f64681cb3f4,stroop,task,task_trial,0,2025-08-16T23:10:42.876Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_0"", ""condition"": ""co...",2025-08-16 23:10:42
1,5,demo,b238e6d7-ec06-461f-a9d4-7f64681cb3f4,stroop,task,task_trial,1,2025-08-16T23:10:44.534Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_1"", ""condition"": ""in...",2025-08-16 23:10:44
2,6,demo,b238e6d7-ec06-461f-a9d4-7f64681cb3f4,stroop,task,task_trial,0,2025-08-16T23:10:46.066Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_0"", ""condition"": ""co...",2025-08-16 23:10:46
3,7,demo,b238e6d7-ec06-461f-a9d4-7f64681cb3f4,stroop,task,task_trial,1,2025-08-16T23:10:47.705Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_1"", ""condition"": ""in...",2025-08-16 23:10:47
4,8,demo,b238e6d7-ec06-461f-a9d4-7f64681cb3f4,stroop,task,task_trial,2,2025-08-16T23:10:49.304Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_2"", ""condition"": ""in...",2025-08-16 23:10:49
...,...,...,...,...,...,...,...,...,...,...,...
276,281,demo,0723f279-7165-4edd-adff-8e7ec31b9d7c,stroop,task,task_trial,5,2025-08-17T00:27:21.341Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_5"", ""condition"": ""co...",2025-08-17 00:27:21
277,282,demo,0723f279-7165-4edd-adff-8e7ec31b9d7c,stroop,task,task_trial,6,2025-08-17T00:27:22.905Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_6"", ""condition"": ""co...",2025-08-17 00:27:22
278,283,demo,0723f279-7165-4edd-adff-8e7ec31b9d7c,stroop,task,task_trial,7,2025-08-17T00:27:24.889Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_7"", ""condition"": ""in...",2025-08-17 00:27:24
279,284,demo,0723f279-7165-4edd-adff-8e7ec31b9d7c,stroop,task,task_trial,8,2025-08-17T00:27:26.763Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_8"", ""condition"": ""in...",2025-08-17 00:27:26


In [53]:
drop_items = ['subject_id', 'session_id', 'start_timestamp']
id_cols = ['subject_id', 'session_id']
app_id = 'behavioral'
app_df = events_df.query("app_id == @app_id").set_index(id_cols)
res_df = (
    app_df['payload'].apply(json.loads).apply(pd.Series)
    .query("item_id not in @drop_items")
    .reset_index()
    .pivot(index=id_cols, columns='item_id', values='value')
)
res_df

item_id                                         alcohol_units  \
subject_id session_id                                           
demo       2a9e9586-aff4-4690-bafe-9a6c112dbb7f             0   
           2f5cadf2-0177-4ff7-831b-30a7e293238a             2   

item_id                                         caffeine_after_noon  \
subject_id session_id                                                 
demo       2a9e9586-aff4-4690-bafe-9a6c112dbb7f                  No   
           2f5cadf2-0177-4ff7-831b-30a7e293238a                 Yes   

item_id                                         deep_work_min_band  \
subject_id session_id                                                
demo       2a9e9586-aff4-4690-bafe-9a6c112dbb7f                NaN   
           2f5cadf2-0177-4ff7-831b-30a7e293238a                 90   

item_id                                         exercise_min_band  \
subject_id session_id                                               
demo       2a9e9586-aff4-4690-bafe-9a6c112dbb7f                10   
           2f5cadf2-0177-4ff7-831b-30a7e293238a                10   

item_id                                         gaming_min_band  \
subject_id session_id                                             
demo       2a9e9586-aff4-4690-bafe-9a6c112dbb7f              30   
           2f5cadf2-0177-4ff7-831b-30a7e293238a              60   

item_id                                         illness_or_travel  \
subject_id session_id                                               
demo       2a9e9586-aff4-4690-bafe-9a6c112dbb7f           Illness   
           2f5cadf2-0177-4ff7-831b-30a7e293238a           Illness   

item_id                                         outdoor_light_min_band  \
subject_id session_id                                                    
demo       2a9e9586-aff4-4690-bafe-9a6c112dbb7f                     20   
           2f5cadf2-0177-4ff7-831b-30a7e293238a                     20   

item_id                                         social_friends_min_band  \
subject_id session_id                                                     
demo       2a9e9586-aff4-4690-bafe-9a6c112dbb7f                      60   
           2f5cadf2-0177-4ff7-831b-30a7e293238a                     120   

item_id                                         social_partner_min_band  
subject_id session_id                                                    
demo       2a9e9586-aff4-4690-bafe-9a6c112dbb7f                      15  
           2f5cadf2-0177-4ff7-831b-30a7e293238a                      60

In [46]:
app_df

,id,subject_id,session_id,app_id,app_type,event_type,event_index,ts_utc,tz,payload,server_ts
12,16,demo,850ea825-891a-42ea-8341-b73a4973b64c,behavioral,survey,survey_item,0,2025-08-16T23:15:45.495Z,America/Los_Angeles,"{""item_id"": ""subject_id"", ""value"": """"}",2025-08-16 23:15:45
13,17,demo,850ea825-891a-42ea-8341-b73a4973b64c,behavioral,survey,survey_item,1,2025-08-16T23:15:45.508Z,America/Los_Angeles,"{""item_id"": ""session_id"", ""value"": """"}",2025-08-16 23:15:45
14,18,demo,850ea825-891a-42ea-8341-b73a4973b64c,behavioral,survey,survey_item,2,2025-08-16T23:15:45.514Z,America/Los_Angeles,"{""item_id"": ""start_timestamp"", ""value"": """"}",2025-08-16 23:15:45
24,28,demo,2f5cadf2-0177-4ff7-831b-30a7e293238a,behavioral,survey,survey_item,0,2025-08-16T23:29:09.792Z,America/Los_Angeles,"{""item_id"": ""caffeine_after_noon"", ""value"": ""Y...",2025-08-16 23:29:09
25,29,demo,2f5cadf2-0177-4ff7-831b-30a7e293238a,behavioral,survey,survey_item,1,2025-08-16T23:29:09.803Z,America/Los_Angeles,"{""item_id"": ""alcohol_units"", ""value"": ""2""}",2025-08-16 23:29:09
26,30,demo,2f5cadf2-0177-4ff7-831b-30a7e293238a,behavioral,survey,survey_item,2,2025-08-16T23:29:09.809Z,America/Los_Angeles,"{""item_id"": ""exercise_min_band"", ""value"": ""10""}",2025-08-16 23:29:09
27,31,demo,2f5cadf2-0177-4ff7-831b-30a7e293238a,behavioral,survey,survey_item,3,2025-08-16T23:29:09.817Z,America/Los_Angeles,"{""item_id"": ""outdoor_light_min_band"", ""value"":...",2025-08-16 23:29:09
28,32,demo,2f5cadf2-0177-4ff7-831b-30a7e293238a,behavioral,survey,survey_item,4,2025-08-16T23:29:09.823Z,America/Los_Angeles,"{""item_id"": ""deep_work_min_band"", ""value"": ""90""}",2025-08-16 23:29:09
29,33,demo,2f5cadf2-0177-4ff7-831b-30a7e293238a,behavioral,survey,survey_item,5,2025-08-16T23:29:09.829Z,America/Los_Angeles,"{""item_id"": ""social_partner_min_band"", ""value""...",2025-08-16 23:29:09
30,34,demo,2f5cadf2-0177-4ff7-831b-30a7e293238a,behavioral,survey,survey_item,6,2025-08-16T23:29:09.834Z,America/Los_Angeles,"{""item_id"": ""social_friends_min_band"", ""value""...",2025-08-16 23:29:09


## Testing

In [ ]:
# import sqlite3
# db = "/Users/guhansundar/Documents/GuData/ObjectiveSubjectiveHealth/data/database.db"
# conn = sqlite3.connect(db)
# cur = conn.cursor()
# sid = 'ce757866-4310-4e0a-97ed-c440e828c774'
# cur.execute("BEGIN")
# cur.execute("DELETE FROM events WHERE session_id=?", (sid,))
# cur.execute("DELETE FROM assets WHERE session_id=?", (sid,))
# cur.execute("DELETE FROM sessions WHERE session_id=?", (sid,))
# conn.commit()
# conn.close()

In [32]:
events_df

,id,subject_id,session_id,app_id,app_type,event_type,event_index,ts_utc,tz,payload,server_ts
0,4,demo,b238e6d7-ec06-461f-a9d4-7f64681cb3f4,stroop,task,task_trial,0,2025-08-16T23:10:42.876Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_0"", ""condition"": ""co...",2025-08-16 23:10:42
1,5,demo,b238e6d7-ec06-461f-a9d4-7f64681cb3f4,stroop,task,task_trial,1,2025-08-16T23:10:44.534Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_1"", ""condition"": ""in...",2025-08-16 23:10:44
2,6,demo,b238e6d7-ec06-461f-a9d4-7f64681cb3f4,stroop,task,task_trial,0,2025-08-16T23:10:46.066Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_0"", ""condition"": ""co...",2025-08-16 23:10:46
3,7,demo,b238e6d7-ec06-461f-a9d4-7f64681cb3f4,stroop,task,task_trial,1,2025-08-16T23:10:47.705Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_1"", ""condition"": ""in...",2025-08-16 23:10:47
4,8,demo,b238e6d7-ec06-461f-a9d4-7f64681cb3f4,stroop,task,task_trial,2,2025-08-16T23:10:49.304Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_2"", ""condition"": ""in...",2025-08-16 23:10:49
...,...,...,...,...,...,...,...,...,...,...,...
277,281,demo,0723f279-7165-4edd-adff-8e7ec31b9d7c,stroop,task,task_trial,5,2025-08-17T00:27:21.341Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_5"", ""condition"": ""co...",2025-08-17 00:27:21
278,282,demo,0723f279-7165-4edd-adff-8e7ec31b9d7c,stroop,task,task_trial,6,2025-08-17T00:27:22.905Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_6"", ""condition"": ""co...",2025-08-17 00:27:22
279,283,demo,0723f279-7165-4edd-adff-8e7ec31b9d7c,stroop,task,task_trial,7,2025-08-17T00:27:24.889Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_7"", ""condition"": ""in...",2025-08-17 00:27:24
280,284,demo,0723f279-7165-4edd-adff-8e7ec31b9d7c,stroop,task,task_trial,8,2025-08-17T00:27:26.763Z,America/Los_Angeles,"{""item_id"": ""stroop_trial_8"", ""condition"": ""in...",2025-08-17 00:27:26


In [33]:
cleaned = expand_by_app(events_df, app_col="app_id", payload_col="payload")

In [34]:
cleaned.keys()

dict_keys(['behavioral', 'daily_core', 'nback_1min_v1', 'pvt_1min_v1', 'stroop', 'wellbeing'])

In [37]:
cleaned['nback_1min_v1']

,id,subject_id,session_id,app_id,app_type,event_type,event_index,ts_utc,tz,server_ts,...,nback_1min_v1__key_pressed,nback_1min_v1__correct,nback_1min_v1__rt_ms,nback_1min_v1__ts_utc,nback_1min_v1__tz,nback_1min_v1__rt,nback_1min_v1__response,nback_1min_v1__trial_type,nback_1min_v1__plugin_version,nback_1min_v1__time_elapsed
42,46.0,demo,48bca019-7188-464c-8a31-711f2d877288,nback_1min_v1,task,task_trial,1.0,2025-08-16T23:30:39.104Z,America/Los_Angeles,2025-08-16 23:30:39,...,space,1.0,252.0,None,America/Los_Angeles,252.0,,html-keyboard-response,2.1.0,35352.0
43,47.0,demo,48bca019-7188-464c-8a31-711f2d877288,nback_1min_v1,task,task_trial,3.0,2025-08-16T23:30:39.911Z,America/Los_Angeles,2025-08-16 23:30:39,...,None,1.0,NaN,None,America/Los_Angeles,NaN,None,html-keyboard-response,2.1.0,36159.0
44,48.0,demo,48bca019-7188-464c-8a31-711f2d877288,nback_1min_v1,task,task_trial,5.0,2025-08-16T23:30:40.716Z,America/Los_Angeles,2025-08-16 23:30:40,...,None,1.0,NaN,None,America/Los_Angeles,NaN,None,html-keyboard-response,2.1.0,36965.0
45,49.0,demo,48bca019-7188-464c-8a31-711f2d877288,nback_1min_v1,task,task_trial,7.0,2025-08-16T23:30:41.522Z,America/Los_Angeles,2025-08-16 23:30:41,...,None,1.0,NaN,None,America/Los_Angeles,NaN,None,html-keyboard-response,2.1.0,1359.0
46,50.0,demo,48bca019-7188-464c-8a31-711f2d877288,nback_1min_v1,task,task_trial,9.0,2025-08-16T23:30:42.330Z,America/Los_Angeles,2025-08-16 23:30:42,...,space,1.0,385.0,None,America/Los_Angeles,385.0,,html-keyboard-response,2.1.0,2050.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,1.0,NaN,None,America/Los_Angeles,NaN,None,html-keyboard-response,2.1.0,24644.0
120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,1.0,NaN,None,America/Los_Angeles,NaN,None,html-keyboard-response,2.1.0,25451.0
121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,1.0,NaN,None,America/Los_Angeles,NaN,None,html-keyboard-response,2.1.0,26258.0
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,1.0,NaN,None,America/Los_Angeles,NaN,None,html-keyboard-response,2.1.0,27065.0


In [22]:
# drop_ids = ['7d7c55cc-cb2a-4fc9-8ecb-e7d9d41d1c80', '7d7c55cc-cb2a-4fc9-8ecb-e7d9d41d1c80']
# events_df.query("session_id not in @drop_ids")
events_df.query("app_type == 'survey'")

,id,subject_id,session_id,app_id,app_type,event_type,event_index,ts_utc,tz,payload,server_ts
12,16,demo,850ea825-891a-42ea-8341-b73a4973b64c,behavioral,survey,survey_item,0,2025-08-16T23:15:45.495Z,America/Los_Angeles,"{""item_id"": ""subject_id"", ""value"": """"}",2025-08-16 23:15:45
13,17,demo,850ea825-891a-42ea-8341-b73a4973b64c,behavioral,survey,survey_item,1,2025-08-16T23:15:45.508Z,America/Los_Angeles,"{""item_id"": ""session_id"", ""value"": """"}",2025-08-16 23:15:45
14,18,demo,850ea825-891a-42ea-8341-b73a4973b64c,behavioral,survey,survey_item,2,2025-08-16T23:15:45.514Z,America/Los_Angeles,"{""item_id"": ""start_timestamp"", ""value"": """"}",2025-08-16 23:15:45
15,19,demo,b3250f07-f82d-408d-9310-26ce1efb496b,wellbeing,survey,survey_item,0,2025-08-16T23:15:54.386Z,America/Los_Angeles,"{""item_id"": ""subject_id"", ""value"": """"}",2025-08-16 23:15:54
16,20,demo,b3250f07-f82d-408d-9310-26ce1efb496b,wellbeing,survey,survey_item,1,2025-08-16T23:15:54.426Z,America/Los_Angeles,"{""item_id"": ""session_id"", ""value"": """"}",2025-08-16 23:15:54
...,...,...,...,...,...,...,...,...,...,...,...
198,202,demo,07a94a20-768e-4b9f-9f57-407139b57d6b,wellbeing,survey,survey_item,4,2025-08-17T00:21:15.345Z,America/Los_Angeles,"{""item_id"": ""wwb_interested"", ""value"": ""5""}",2025-08-17 00:21:15
199,203,demo,07a94a20-768e-4b9f-9f57-407139b57d6b,wellbeing,survey,survey_item,5,2025-08-17T00:21:15.350Z,America/Los_Angeles,"{""item_id"": ""wst_unexpected"", ""value"": ""4""}",2025-08-17 00:21:15
200,204,demo,07a94a20-768e-4b9f-9f57-407139b57d6b,wellbeing,survey,survey_item,6,2025-08-17T00:21:15.354Z,America/Los_Angeles,"{""item_id"": ""wst_control"", ""value"": ""0""}",2025-08-17 00:21:15
201,205,demo,07a94a20-768e-4b9f-9f57-407139b57d6b,wellbeing,survey,survey_item,7,2025-08-17T00:21:15.359Z,America/Los_Angeles,"{""item_id"": ""wst_confident"", ""value"": ""1""}",2025-08-17 00:21:15


In [ ]:
DATABASE_PATH = '/Users/guhansundar/Documents/GuData/ObjectiveSubjectiveHealth/data/database.db'
conn = sqlite3.connect(DATABASE_PATH)


subject_id = 'demo'

# Query the data
query = f"""
SELECT *
FROM events
WHERE app_type = 'survey'
and subject_id = '{subject_id}'
"""
df = pd.read_sql_query(query, conn)

In [ ]:
df

In [ ]:
print(list_available_surveys(conn, subject_id='demo'))

In [6]:
df = get_survey_data_for_subject(subject_id='demo', survey='daily_core', conn=conn)


In [7]:
df

,energy_1to5__value,happiness_1to5__value,productive_1to5__value,session_id__value,social_fulfillment_1to5__value,start_timestamp__value,stress_1to5__value,subject_id__value,ts_utc
session_id,,,,,,,,,
2025-08-16 23:10:23.683000+00:00,5.0,3.0,2.0,NaN,3.0,NaN,5.0,NaN,2025-08-17 00:20:42.629000+00:00


In [ ]:
print("Current PYTHONPATH:", sys.path)

In [ ]:
print("Current PYTHONPATH:", sys.path)

In [5]:
wide

,subject_id,session_id,app_id,app_type,ts_utc,happiness_1to5__value,energy_1to5__value,stress_1to5__value,productive_1to5__value,social_fulfillment_1to5__value
0,demo,2025-08-16 08:41:20.483000+00:00,daily_core,survey,2025-08-16 21:12:58.641000+00:00,4.0,5.0,3.0,5.0,2.0
